<h3>Ankit Bansal Complex SQL 2 | find new and repeat customers</h3>

<h2>query data set</h2>

In [0]:
%sql 
drop table customer_orders;
create table customer_orders (
order_id integer,
customer_id integer,
order_date date,
order_amount integer
);
insert into customer_orders values
(1,100,cast('2022-01-01' as date),2000),
(2,200,cast('2022-01-01' as date),2500),
(3,300,cast('2022-01-01' as date),2100),
(4,100,cast('2022-01-02' as date),2000),
(5,400,cast('2022-01-02' as date),2200),
(6,500,cast('2022-01-02' as date),2700),
(7,100,cast('2022-01-03' as date),3000),
(8,400,cast('2022-01-03' as date),1000),
(9,600,cast('2022-01-03' as date),3000)
;

num_affected_rows,num_inserted_rows
9,9


In [0]:
%sql
select * from customer_orders

order_id,customer_id,order_date,order_amount
1,100,2022-01-01,2000
2,200,2022-01-01,2500
3,300,2022-01-01,2100
4,100,2022-01-02,2000
5,400,2022-01-02,2200
6,500,2022-01-02,2700
7,100,2022-01-03,3000
8,400,2022-01-03,1000
9,600,2022-01-03,3000


<h2>Solution in sql(hive)</h2>

In [0]:
%sql 
select c1.order_date, 
sum(case 
  when c1.order_date=c2.fst_dt then 1 else 0 end) as first_vist,
sum(case 
  when c1.order_date!=c2.fst_dt then 1 else 0 end) as repeate_vist
 from customer_orders c1
left join (
  select customer_id,min(order_date) as fst_dt from  customer_orders group by customer_id
) c2 on c1.customer_id=c2.customer_id
group by c1.order_date


order_date,first_vist,repeate_vist
2022-01-03,1,2
2022-01-01,3,0
2022-01-02,2,1


<h2>Solution in Pyspark</h2>

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.functions import min
from pyspark.sql.functions import when
from pyspark.sql.functions import sum

In [0]:
# put data in pyspark data frame from the hive table
df=spark.sql("select * from customer_orders")
df.show()

+--------+-----------+----------+------------+
|order_id|customer_id|order_date|order_amount|
+--------+-----------+----------+------------+
|       1|        100|2022-01-01|        2000|
|       2|        200|2022-01-01|        2500|
|       3|        300|2022-01-01|        2100|
|       4|        100|2022-01-02|        2000|
|       5|        400|2022-01-02|        2200|
|       6|        500|2022-01-02|        2700|
|       7|        100|2022-01-03|        3000|
|       8|        400|2022-01-03|        1000|
|       9|        600|2022-01-03|        3000|
+--------+-----------+----------+------------+



In [0]:
df_fst=df.groupBy("customer_id").agg(min("order_date").alias("fst_dt"))
df_vist=df.join(df_fst,df.customer_id==df_fst.customer_id,"inner").withColumn("first_vist",\
    when(df["order_date"]==df_fst["fst_dt"],1).otherwise(0))\
    .withColumn("repeate_vist",\
    when(df["order_date"]!=df_fst["fst_dt"],1).otherwise(0))
df_final=df_vist.groupBy("order_date").agg(sum("first_vist").alias("first_vist"),\
    sum("repeate_vist").alias("repeate_vist"))
df_final.show()

+----------+----------+------------+
|order_date|first_vist|repeate_vist|
+----------+----------+------------+
|2022-01-03|         1|           2|
|2022-01-01|         3|           0|
|2022-01-02|         2|           1|
+----------+----------+------------+

